### Implement logging in machine learning systems

In [2]:
import logging

# Set up logging to a file
logging.basicConfig(filename='ml_pipeline.log', level=logging.INFO)

# Example log message
logging.info("Logging setup complete.")

In [21]:
import pandas as pd
import numpy

# Log the start of data loading
logging.info("Loading dataset...")

# Load the dataset
# df = pd.read_csv('your-dataset.csv')
df = pd.DataFrame(numpy.random.randint(1,100,100))
logging.info("Dataset loaded successfully.")

# Log the start of preprocessing
logging.info("Starting data preprocessing...")

# Example preprocessing: handling missing values
df.fillna(0, inplace=True)
logging.info("Missing values filled with 0.")

# Log the completion of preprocessing
logging.info("Data preprocessing completed.")

In [22]:
logging.error("Example error logged")

### AzureML deployement example

In [87]:
import azureml.core as core
core.VERSION

'1.60.0'

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

In [70]:
import dill
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge

diab_x , diab_y = load_diabetes(return_X_y=True)

model = Ridge()
model.fit(diab_x, diab_y)

dill.dump(model, open("mymodel.pkl", "wb"))

In [72]:
import numpy as np

np.savetxt("diab_x.csv", x, delimiter=",")
np.savetxt("diab_y.csv", y, delimiter=",")

In [ ]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
datastore.upload_files(files=["./diab_x.csb", "./diab_y.csv"],
                       target_path="./diab_regression", overwrite=True)
data_x = Dataset.Tabular.from_delimited_files(path = [(datastore, "diab_regression/diab_x.csv")])
data_y = Dataset.Tabular.from_delimited_files(path = [(datastore, "diab_regression/diab_y.csv")])


In [ ]:
import sklearn
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

cloud_model = Model.register(workspace=ws,
                             model_name="MyCloudModel",
                             model_path="./mymodel.pkl",
                             model_framework=Model.Framework.SCIKITLEARN,
                             model_framework_version=sklearn.__version__,
                             sample_input_dataset=diab_x,
                             sample_output_dataset=diab_y,
                             resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                             description="Ridge predictor model of diabetes",
                             tags={"area":"diabetes", "type":"Ridge Regression"}
                             )
print("Name: ", cloud_model.name)
print("Version: ", cloud_model.version)


In [82]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("my-diab-env")
env.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    "pip==20.2.4"],
    pip_packages=[
        "azureml-defaults",
        "inferences-schema[numpy-support]",
        "joblib",
        "dill==0.3.6",
        "numpy==1.23",
        f"scikit-learn=={sklearn.__version__}"
])

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

service_name = "my-custom-env-service"
inference_config = InferenceConfig(entry_script="model.py", environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=0.5)
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[cloud_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)